In [ ]:
class Scrapper():
    def __init__(self):
        self.aka = 0
        
# This funtion will put the company name in the search field and press ENTER.
# After this it'll search for the comapny name in the list appeared
    def go_to_site(self, company_name):
        
        # Put company name in search firld and press ENTER
        search = browser.find_element_by_name("q")
        try :
            search.send_keys(company_name)
        except WebDriverException as exception:
            pass
    
        search.send_keys(Keys.ENTER)
        
        # Give it some time to load the list of related information
        time.sleep(3)
        
        # Select the company name matching to the one we are searching for
        related_info = browser.find_elements_by_class_name("title")
        for name in related_info:    
            try:
                if company_name.lower() in name.text.lower():
                    name.click()
                    break
                else:
                    continue

            except StaleElementReferenceException:
                continue
                
   
   # This funtion'll extract the required info like "Location",'Specialities','Company size','Website' 
    # from web page and save it into dictionary             
    def get_company_info(self, c_info):
        col = ["Location",'Specialities','Company size','Website']
        info = {}
        a = c_info.split('\n')
    
        for i in range(len(a)):
            info[col[i]] = a[i]
        
        # It is capturing Domain name from website
        if "Website" in info.keys(): 
            for i in (info["Website"]).split('.'):
                if len(i) > 5 :
                    info["Domain_name"] = i
        else:
            pass
        return info
                
    # This will extract name of all the investors investing in the company    
    def get_investors(self,Investors):        
        investors = {}        
        l = Investors.split("\n")
        inv = ["Inverstor_1", "Inverstor_2","Inverstor_3","Inverstor_4", "Inverstor_5","Inverstor_6"]
        k = 0
        
        # To avoid extracting their professions, select words with 
        # length greater than or equal to 2 and less than or equal to 3
        for i in range(len(l)):
            if len(l[i].split(" ")) >= 2:
                j = l[i].split(" ")

                if len(j[0]) > 3 and len(j[1]) > 3:
                    investors[inv[k]] = l[i]
                    k =+ 1
                else:
                    pass
        return investors
    
    # This will get info of all the founders of the company
    def get_founders(self, founders):
        founder = {}
        f = founders.split('\n') 
        fdr = []
        for i in range(len(f)):
            if len(f[i].split(' ')) >= 2 and len(f[i].split(' ')) <= 3:
                fdr.append(f[i])
            else:
                pass
        fdr_list = ["Founder_1", "Founder_2", "Founder_3", "Founder_4", "Founder_5","Founder_6", "Founder_7", "Founder_8", "Founder_9", "Founder_10"]

        for i in range(len(fdr)):
            founder[fdr_list[i]] = fdr[i]
        return founder
    
    # This will extract sum of all the fundings the company is receiving
    def get_fundings(self, funds):
        value = 0
        for i in funds:
            try : 
                j = int(i.text.replace(',', '')[1:])
                value = value + j
            except ValueError as exception:
                pass
        return value
    
    # This funtion will save data into csv file
     def dict_to_csv(self, scraped_values_dict): 
        my_dict = scraped_values_dict
        with open('Data_1.csv', 'a') as f:
            w = csv.DictWriter(f, col_c_info)
            if f.tell() == 0:
                w.writeheader()
                w.writerow(my_dict)
            else: 
                w.writerow(my_dict)
                
                
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import getpass
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, WebDriverException
import csv
import time
import pandas as pd
import numpy as np


chromedriver = '/usr/bin/chromedriver'
browser = webdriver.Chrome(chromedriver)

# Open the site
browser.get('https://angel.co/login')

# Provide username and password
username = browser.find_element_by_id("user_email")
password = browser.find_element_by_id("user_password")

username.send_keys("akankshakaple007@gmail.com")
password.send_keys("@k@nksh@007")

browser.find_element_by_class_name("c-button").click()

# Provide the list of compaanies to extract the info
data = pd.read_csv("Data_with_result.csv")
company_name = np.array(data["Company_Name"])

# Elements we want from the website into csv file
col_c_info = ["Company_name",'Website',"Domain_name","Location",'Specialities','Company size'
              ,"LinkedIn_URL","Funding" ,"Founder_1", "Founder_2", "Founder_3", "Founder_4", "Founder_5"
              ,"Founder_6","Founder_7", "Founder_8", "Founder_9", "Founder_10"
              ,"Inverstor_1", "Inverstor_2","Inverstor_3","Inverstor_4", "Inverstor_5","Inverstor_6"
              ,"Job_1", "Job_2", "Job_3", "Job_4", "Job_5", "Job_6", "Job_7", "Job_8", "Job_9","Job_10"]


# Create an object for class Scraper
s = Scrapper()

for i in range(2007, 2300):
    print(i)  # Print value for verification
    
    c_info = {}
    investors = {}
    founders = {}
    dict_1 = {}
    
    print(company_name[i]) # Print current company name for verification
    
    # As company name might contain some extra words, so remove them
    cn = data["Company_Name"][i]
    l = ['Pvt.', 'Ltd.', "Private", "Limited", "Pvt", "Ltd"]
    for j in range(len(l)):
        if l[j] in cn:
            cn = cn.replace(l[j], ' ')
            print(cn)
        
    dict_1["Company_name"] = company_name[i]
    
    try :
        s.go_to_site(cn)
        
    except StaleElementReferenceException as exception:
        pass
    except TimeoutException as exception:
        pass
    except WebDriverException as exception:
        pass
    
    time.sleep(5)
    
    try :     
        c_info = s.get_company_info(browser.find_element_by_class_name('section_container').text)
        
    except NoSuchElementException as exception:
        try :
            s.go_to_site(cn)
            time.sleep(5)
            c_info = s.get_company_info(browser.find_element_by_class_name('section_container').text)
            
        except NoSuchElementException as exception:
            pass
    
    try :
        linkedin_url = browser.find_element_by_class_name('linkedin_url').get_attribute('href')
        dict_1["LinkedIn_URL"] = linkedin_url
    except NoSuchElementException as exception:
        pass
    
    try : 
        founders = s.get_founders(browser.find_element_by_class_name('startup_profile_group').text)
    except NoSuchElementException as exception:
        pass
    
    try :
        investors = s.get_investors(browser.find_element_by_class_name('medium').text)
    except NoSuchElementException as exception:
        pass
    
    try :
        dict_1["Funding"] = s.get_fundings(browser.find_elements_by_class_name("raised"))
    except NoSuchElementException as exception:
        pass
    
    # Go to the company's job application page to get all the jobs company is offering 
    # related to software
    try :
        browser.find_element_by_class_name("u-widthFull").click()
    except NoSuchElementException as exception:
        pass
    
    time.sleep(3)
    

    jobs = {}
       
    k = 0
    j = []   
    
    # This class name differs from company to company 
    try : 

        jobs_avail = browser.find_element_by_class_name("s-grid-colMd18").text
        j_list = jobs_avail.split("\n")
        k = 0
        j = ["Job_1", "Job_2", "Job_3", "Job_4", "Job_5", "Job_6", "Job_7", "Job_8", "Job_9","Job_10"]
        l = j_list[::3]
        l = l[:10]
        for i in l:
            jobs[j[k]] = i
            k = k + 1
        dict_1 = {**dict_1, **jobs}
        
    except NoSuchElementException as exception:
        
        try :
            jobs_avail = browser.find_element_by_class_name("listing-title").text
            j_list = jobs_avail.split("\n")
            k = 0
            j = ["Job_1", "Job_2", "Job_3", "Job_4", "Job_5", "Job_6", "Job_7", "Job_8", "Job_9","Job_10"]
            l = j_list[::3]
            l = l[:10]
            for i in l:
                jobs[j[k]] = i
                k = k + 1
            dict_1 = {**dict_1, **jobs}
            
        except NoSuchElementException as exception:
            pass
        
    # After extracting info of jobs, go back to the home page    
    browser.back()     
    
    # Combine all the info into a dictionary
    my_dict = {**c_info, **investors, **founders, **dict_1}   
           
    # Convert dictionary into csv file
    s.dict_to_csv(my_dict)
    

2007
Kartrocket
2008
ACS Global Tech Solutions Pvt Ltd
ACS Global Tech Solutions   Ltd
ACS Global Tech Solutions    
2009
BrickRed Technologies
2010
emPhi
2011
IISGL
2012
Appinventiv
2013
Shriv Commedia Solutions Pvt Ltd
Shriv Commedia Solutions   Ltd
Shriv Commedia Solutions    
2014
CodeIgnition
2015
Yapapaya Technologies LLP
2016
Astute Systems Technology Private Limited
Astute Systems Technology   Limited
Astute Systems Technology    
2017
S-EDGE Software
2018
Cross Byrne Design Studio Pvt. Ltd.
Cross Byrne Design Studio   Ltd.
Cross Byrne Design Studio    
2019
HappyShappy
2020
IndyLogix Solutions
2021
Stigasoft
2022
Admiral Technologies
2023
Bhramm Technologies
2024
Weaverbird Engineering & Technology Pvt Ltd
Weaverbird Engineering & Technology   Ltd
Weaverbird Engineering & Technology    
2025
M1 LOGIX TECHNOLOGIES
2026
jusTap
2027
Depasser Infotech
2028
Future Next Technologies
2029
Ariel Software Solutions Pvt. Ltd.
Ariel Software Solutions   Ltd.
Ariel Software Solutions    
